https://scikit-learn.org/stable/modules/preprocessing.htm

In [2]:
# -*- coding: utf-8 -*-
import os
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import check_random_state
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler 
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from termcolor import colored as cl
np.random.seed(31337)

In [3]:
data=pd.read_csv("./data_transformed.csv")
data.head()

,Unnamed: 0,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,T21,T22,T23,T24,T25,T26,T27,T28,value,Class
0,169876,-0.611712,-0.769705,-0.149759,-0.224877,2.028577,-2.019887,0.292491,-0.523020,0.358468,...,-0.075208,0.045536,0.380739,0.023440,-2.220686,-0.201146,0.066501,0.221180,1.79,0
1,127467,-0.814682,1.319219,1.329415,0.027273,-0.284871,-0.653985,0.321552,0.435975,-0.704298,...,-0.128619,-0.368565,0.090660,0.401147,-0.261034,0.080621,0.162427,0.059456,1.98,0
2,137900,-0.318193,1.118618,0.969864,-0.127052,0.569563,-0.532484,0.706252,-0.064966,-0.463271,...,-0.305402,-0.774704,-0.123884,-0.495687,-0.018148,0.121679,0.249050,0.092516,0.89,0
3,21513,-1.328271,1.018378,1.775426,-1.574193,-0.117696,-0.457733,0.681867,-0.031641,0.383872,...,-0.220815,-0.419013,-0.239197,0.009967,0.232829,0.814177,0.098797,-0.004273,15.98,0
4,134700,1.276712,0.617120,-0.578014,0.879173,0.061706,-1.472002,0.373692,-0.287204,-0.084482,...,-0.160161,-0.430404,-0.076738,0.258708,0.552170,0.370701,-0.034255,0.041709,0.76,0


In [4]:
data.describe()

,Unnamed: 0,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,T21,T22,T23,T24,T25,T26,T27,T28,value,Class
count,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,...,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000,281959.000000
mean,142415.251643,-0.000054,0.000040,-0.000128,-0.000310,-0.000222,-0.000069,-0.000197,0.000059,0.000007,...,-0.000216,-0.000016,-0.000141,-0.000092,0.000029,0.000059,-0.000062,-0.000110,88.335340,0.001738
std,82208.247997,1.959208,1.651774,1.516882,1.415975,1.381363,1.332312,1.238870,1.195670,1.098890,...,0.733862,0.725659,0.625375,0.605570,0.521293,0.482182,0.404187,0.329206,250.290644,0.041651
min,0.000000,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-43.557242,-73.216718,-13.434066,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.000000,0.000000
25%,71237.500000,-0.920600,-0.598572,-0.890549,-0.848853,-0.691687,-0.767719,-0.554134,-0.208488,-0.643141,...,-0.228425,-0.542624,-0.161856,-0.354748,-0.317061,-0.326992,-0.070840,-0.052969,5.600000,0.000000
50%,142410.000000,0.018145,0.065234,0.179850,-0.020404,-0.054246,-0.274187,0.039920,0.022395,-0.051416,...,-0.029499,0.007105,-0.011210,0.041014,0.016554,-0.051901,0.001321,0.011226,22.000000,0.000000
75%,213601.500000,1.315615,0.803611,1.027013,0.742886,0.611863,0.398574,0.570230,0.327472,0.596980,...,0.186207,0.528501,0.147697,0.439415,0.350762,0.240924,0.091077,0.078256,77.210000,0.000000
max,284806.000000,2.454930,22.057729,9.382558,16.875344,34.801666,73.301626,120.589494,20.007208,15.594995,...,27.202839,10.503090,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,1.000000


In [5]:
# Decorators
from functools import wraps


def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper


def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [6]:
def load_data():
    data = pd.read_csv("./data_transformed.csv")
    X = data.drop('Class', axis = 1).values
    y = data['Class'].values
    return(X,y)
   # self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
   # X, y, test_size=0.65, random_state=42)

In [7]:
 class Normalize(object): 
    def normalize(self, X_train, X_test):
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_test  = self.scaler.transform(X_test)
        return (X_train, X_test) 
    
    def inverse(self, X_train, X_val, X_test):
        X_train = self.scaler.inverse_transform(X_train)
        X_test  = self.scaler.inverse_transform(X_test)
        return (X_train, X_test)   

def split(X,y, splitRatio):
    X_train = X[:splitRatio]
    y_train = y[:splitRatio]
    X_test = X[splitRatio:]
    y_test = y[splitRatio:]
    return (X_train, y_train, X_test, y_test) 

In [8]:
  class TheAlgorithm(object):
  
    @my_logger
    @my_timer
    def __init__(self, X_train, y_train, X_test, y_test): 
        self.X_train, self.y_train, self.X_test, self.y_test = X_train, y_train, X_test, y_test    
        
    @my_logger
    @my_timer
    def fit(self): 
        normalizer = Normalize()
        self.X_train, self.X_test = normalizer.normalize(self.X_train, self.X_test)   
        train_samples = self.X_train.shape[0]
        self.classifier = svm.SVC(
            C=1.0, kernel='rbf', degree=3, gamma='scale')
        self.classifier.fit(self.X_train, self.y_train)
        self.train_y_predicted = self.classifier.predict(self.X_train)
        self.train_accuracy = np.mean(self.train_y_predicted.ravel() == self.y_train.ravel()) * 100
        self.train_confusion_matrix = confusion_matrix(self.y_train, self.train_y_predicted)        
        return self.train_accuracy
    
    @my_logger
    @my_timer
    def predict(self):
        self.test_y_predicted = self.classifier.predict(self.X_test) 
        self.test_accuracy = np.mean(self.test_y_predicted.ravel() == self.y_test.ravel()) * 100 
        self.test_confusion_matrix = confusion_matrix(self.y_test, self.test_y_predicted)        
        self.report = classification_report(self.y_test, self.test_y_predicted)
        print("Classification report for classifier:\n %s\n" % (self.report))
        return self.test_accuracy

In [9]:
#The solution
if __name__ == '__main__': 
    X,y = load_data()
    print ('data:', X.shape, y.shape)
  
    splitRatio = 60000
    X_train, y_train, X_test, y_test = split(X,y,splitRatio) 

    np.random.seed(31337)
    ta = TheAlgorithm(X_train, y_train, X_test, y_test)
    
    train_accuracy = ta.fit()
    print()
    print('Train Accuracy:', train_accuracy,'\n') 
    print("Train confusion matrix:\n%s\n" % ta.train_confusion_matrix)
  
    test_accuracy = ta.predict()
    print()
    print('Test Accuracy:', test_accuracy,'\n') 
    print("Test confusion matrix:\n%s\n" % ta.test_confusion_matrix)
    
  
  

data: (281959, 30) (281959,)
__init__ ran in: 1.9073486328125e-06 sec
fit ran in: 1.0266518592834473 sec

Train Accuracy: 99.93833333333333 

Train confusion matrix:
[[59898    13]
 [   24    65]]

Classification report for classifier:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    221558
           1       0.86      0.79      0.82       401

    accuracy                           1.00    221959
   macro avg       0.93      0.90      0.91    221959
weighted avg       1.00      1.00      1.00    221959


predict ran in: 1.5376169681549072 sec

Test Accuracy: 99.93827688897498 

Test confusion matrix:
[[221505     53]
 [    84    317]]



In [11]:
class TestInput(unittest.TestCase):
  
    @classmethod
    def setUpClass(cls):
        # print('setupClass')   
        pass

    @classmethod
    def tearDownClass(cls): 
        # print('teardownClass')
        pass

    def setUp(self):
        print('setUp') 
        
        X, y = load_data()
        splitRatio = 60000
        self.X_train, self.y_train, self.X_test, self.y_test = split(X,y,splitRatio) 
        self.train_accuracy = 72.92166666666667
        self.train_confusion_matrix = np.array([[5447,   5,  40,  31,  49,  16, 198,  50,  81,   6],
                                                 [   3,6440, 127,  54,   3,  29,  25,  36,  24,   1],
                                                 [ 297, 420,3824, 163, 256,  19, 622, 186, 121,  50],
                                                 [ 124, 221, 255,4566,  54, 251,  97, 129, 275, 159],
                                                 [ 104, 128,  26,  54,4546, 342, 206, 133,  96, 207],
                                                 [ 399, 200, 109,1081, 416,2227, 289, 363, 228, 109],
                                                 [ 173,  89, 112,  55, 156, 229,5034,  25,  45,   0],
                                                 [ 213, 192, 205,  39, 160,  17,  26,5058,  60, 295],
                                                 [  67, 690, 202, 677,  73, 188, 347,  39,3437, 131],
                                                 [ 164, 162,  63, 290, 669, 279, 122, 735, 291,3174]])
        self.test_accuracy = 73.4
        self.test_confusion_matrix = np.array([[ 923,   1,   2,   3,   3,   1,  35,   3,   9,   0],
                                                [   0,1084,  23,  11,   0,   0,   5,   4,   8,   0],
                                                [  63,  78, 669,  27,  38,   2,  97,  28,  24,   6],
                                                [  20,  27,  35, 770,   8,  42,  18,  27,  45,  18],
                                                [  15,  21,   3,   8, 750,  60,  45,  23,  18,  39],
                                                [  56,  24,  15, 193,  73, 362,  56,  58,  38,  17],
                                                [  35,  10,  18,  11,  28,  42, 799,   6,   8,   1],
                                                [  23,  40,  52,   6,  21,   4,   7, 821,   8,  46],
                                                [  14,  90,  29,  99,  10,  33,  66,   7, 598,  28],
                                                [  21,  27,  10,  37, 133,  42,  27, 100,  48, 564]])

    def tearDown(self):
        # print('tearDown')
        pass
        
    def test_fit(self):     
        np.random.seed(31337)
        self.ta = TheAlgorithm(self.X_train, self.y_train, self.X_test, self.y_test)
        self.assertTrue(self.ta.fit(), self.train_accuracy) 
        self.assertTrue(self.ta.train_confusion_matrix.tolist(), self.train_confusion_matrix.tolist())  
  
    def test_predict(self):
        np.random.seed(31337)
        self.ta = TheAlgorithm(self.X_train, self.y_train, self.X_test, self.y_test)
        self.ta.fit()
        self.assertTrue(self.ta.predict(), self.test_accuracy)
        self.assertTrue(self.ta.train_confusion_matrix.tolist(), self.train_confusion_matrix.tolist()) 
      
      
if __name__ == '__main__':
  
    #run tests 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

setUp
__init__ ran in: 1.9073486328125e-06 sec


.

fit ran in: 1.0494258403778076 sec
setUp
__init__ ran in: 4.0531158447265625e-06 sec
fit ran in: 1.016341209411621 sec


.

Classification report for classifier:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    221558
           1       0.86      0.79      0.82       401

    accuracy                           1.00    221959
   macro avg       0.93      0.90      0.91    221959
weighted avg       1.00      1.00      1.00    221959


predict ran in: 1.5051393508911133 sec



----------------------------------------------------------------------
Ran 2 tests in 5.539s

OK
